## DeepSEE: Deep Disentangled Semantic Explorative Extreme Super-Resolution

This is a demo for our paper **Deep Disentangled Semantic Explorative Extreme Super-Resolution** ([ACCV 2020](http://accv2020.kyoto/) oral). 

Please check out our [project page](https://mcbuehler.github.io/DeepSEE/) for details and the paper.

## Setup
You can follow the steps below to set up the demo from scratch. 

Make sure to use a GPU runtime:
`Runtime -> Change runtime type -> GPU`

In [ ]:
!git clone https://github.com/mcbuehler/DeepSEE

Cloning into 'DeepSEE'...
remote: Enumerating objects: 357, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 357 (delta 132), reused 307 (delta 87), pack-reused 0
Receiving objects: 100% (357/357), 6.34 MiB | 18.98 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [ ]:
!pip install -r DeepSEE/requirements.txt;

     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 1.9MB 17.2MB/s 
     |████████████████████████████████| 778kB 42.1MB/s 
     |████████████████████████████████| 28.7MB 112kB/s 
     |████████████████████████████████| 348kB 51.5MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.0-cp36-cp36m-linux_x86_64.whl size=67508 sha256=8550530dc4816ea8b5db49a871f00fb1cc21c323759b7f3d53b461fdfa8b3b2e
  Stored in directory: /root/.cache/pip/wheels/54/f9/95/099544e9f879f719b14cf567fabb5aa7984263df0f025f3eef
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=4f3ada1c562bb778837206793cadc66c70301d701b1110012b1569410f4dae0a
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built wrapt GPUtil
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ERROR: google-colab 1.0.0 has 

If you are asked to restart your runtime, please do so.

In [ ]:
import os
os.chdir("DeepSEE")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Add** this folder to your Google Drive. It contains the pre-trained checkpoints and some ready-to-use data samples.

https://drive.google.com/drive/folders/1pVwxpln-lame79yGYPgFA1KvV2iXT_I2?usp=sharing

In [ ]:
from demo import Demo, get_demo_options, display_result
import torch
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

base_path = "/content/gdrive/My Drive/DeepSEE_data/demo_data/"
checkpoints_dir = "/content/gdrive/My Drive/DeepSEE_data/checkpoints"

## Method

A low-resolution input ($x_{lr} \in \mathbb{R}^{H_{lr}\times{W_{lr}}\times{3}}$) image acts as a starting point that carries the low-frequency information. A generator ($G_{\Theta}$) upscales this image and hallucinates the high-frequencies yielding the high-resolution image $\hat x_{hr} \in \mathbb{R}^{H_{hr}\times{W_{hr}}\times{3}}$. As a guidance, $G_{\Theta}$ leverages both a high resolution semantic map ($M \in \mathbb{R}^{H_{hr}\times{W_{hr}}\times{N}}$, where $N$ is the number of the semantic regions) and independent styles per region ($S\in\mathbb{R}^{N\times d}$, where $d$ is the style dimensionality). The upscaled image should thus retain the low-frequency information from the low-resolution image. In addition, it should be consistent in terms of the semantic regions and have specific, yet independent styles per region. We formally define our problem as

$\hat x_{hr} = G_{\Theta}(x_{lr}, \thinspace M, \thinspace S)$

A user is able to control the __appearance__ and __shape__ of each semantic region. 

## Example Results on 8x Upscaling

### Inference for the __Default__ Solution
We upscale a low-resolution (16x16) image to a high-resolution (128x128). We call this the __default solution__.

We provide these sample images from the [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset (bicubically downscaled to 16x16 pixels):

In [ ]:
dataset = "CelebA"
os.listdir(os.path.join(base_path, dataset, "image_16x16"))

['185232.png',
 '195804.png',
 '199152.png',
 '194600.png',
 '201892.png',
 '183306.png',
 '197579.png']

For the __default solution__, the model requires two inputs:
1. the low-resolution image $x_{lr}$ and 
2. a semantic mask $M$ (we predicted the semantic mask from the low-resolution image)

The style matrix $S$ is predicted from the low-resolution image $x_{lr}$.

In [ ]:
opt = get_demo_options("8x_independent_128x128")
opt.checkpoints_dir = checkpoints_dir
demo = Demo(opt)

In [ ]:
filename = "185232.png"

kwargs = {
    "name": "demo",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "predicted_labels", filename)
}
result_default_solution = demo.run(**kwargs)
display_result(result_default_solution, size=(128, 128))

Encoding style from LR image...
Style computed.
Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…

### Style Manipulations
We can add random noise to generate multiple high-resolution variants. 

* Experiment by changing `delta` to make the effect stronger or weaker.
* Choose to influence other/multiple semantic regions by changing/adding indices to `region`.
* The style matrix (or rows thereof) can be computed from other high-resolution images (examples in the paper).
* Rows in the style matrix can be interpolated (examples in the paper).

Here is an overview of all available regions:

In [ ]:
for region_id in range(19):
    print("{:2d}: {}".format(region_id, get_celebA_region_name(region_id)))

 0: Background
 1: Skin
 2: Nose
 3: Eyeglass
 4: Left eye
 5: Right eye
 6: Left eyebrow
 7: Right eyebrow
 8: Left Ear
 9: Right Ear
10: Mouth
11: Upper Lip
12: Lower Lip
13: Hair
14: Hat
15: Earring
16: Necklace
17: Neck
18: Cloth


In the following example, we change the __lips__ adding noise to the corresponding rows in the style matrix.

In [ ]:
encoded_style_orig = result_default_solution["encoded_style"].detach().clone()
encoded_style_noisy = encoded_style_orig.clone()

delta = 0.03
region = [11, 12]  # Corresponds to the lips.

# We add the same noise to both the upper and lower lip
noise = torch.rand([1, encoded_style_orig.shape[2]]) * delta
encoded_style_noisy[:, region, :] = (encoded_style_orig[:, region, :] + torch.stack([noise, noise], dim=1)).clamp(-1, 1)

kwargs = {
    "name": "demo_style_manipulation",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "predicted_labels", filename),
    "encoded_style": encoded_style_noisy
}
result_stochastic = demo.run(**kwargs)
display_result(result_stochastic, size=(128, 128))

Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…

Now let's also modify the __eyebrows__.

In [ ]:
encoded_style_orig = result_default_solution["encoded_style"].detach().clone()
encoded_style_noisy = encoded_style_orig.clone()

delta = 0.03
region = [6, 7]  # Corresponds to the eyesbrows.

noise = torch.rand([1, encoded_style_orig.shape[2]]) * delta
encoded_style_noisy[:, region, :] = (encoded_style_orig[:, region, :] + torch.stack([noise, noise], dim=1)).clamp(-1, 1)

kwargs = {
    "name": "demo_style_manipulation",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "predicted_labels", filename),
    "encoded_style": encoded_style_noisy
}
result_stochastic = demo.run(**kwargs)
display_result(result_stochastic, size=(128, 128))

Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…

## Example Results on 32x Upscaling

We also provide a model for extreme upscaling from 16x16 to 512x512 pixels.

Again, we provide ready-to-use examples. These sample are from the [CelebAMask-HQ](https://github.com/switchablenorms/CelebAMask-HQ) dataset (bicubically downscaled to 16x16 pixels):

In [ ]:
dataset = "CelebAMask-HQ"
os.listdir(os.path.join(base_path, dataset, "image_16x16"))

['14337.png', '28368.png', '19776.png']

In [ ]:
opt = get_demo_options("32x_independent")
opt.checkpoints_dir = checkpoints_dir
demo_32x = Demo(opt)

In [ ]:
filename = "14337.png"

kwargs = {
    "name": "demo",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "predicted_labels", filename)
}
result_default_solution = demo_32x.run(**kwargs)
display_result(result_default_solution)

Encoding style from LR image...
Style computed.
Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…

### Semantic Manipulations
We can repaint the semantic mask and run inference again. This yields different shapes in the upscaled image, but preserves the overall appearance.

In [ ]:
kwargs = {
    "name": "demo_semantic_manipulation",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "manipulated_labels", filename)
}
result_default_solution = demo_32x.run(**kwargs)
display_result(result_default_solution)

Encoding style from LR image...
Style computed.
Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…

### Style Manipulations

In [ ]:
encoded_style_orig = result_default_solution["encoded_style"].detach().clone()
encoded_style_noisy = encoded_style_orig.clone()
import torch
delta = 0.15
region = [1]  # Corresponds to the skin region.

encoded_style_noisy[:, region, :] = (encoded_style_orig[:, region, :] + (torch.rand(encoded_style_orig[:, region, :].shape) * delta)).clamp(-1, 1)
kwargs = {
    "name": "demo_style_manipulation",
    "path_image_lr": os.path.join(base_path, dataset, "image_16x16", filename),
    "path_semantics": os.path.join(base_path, dataset, "manipulated_labels", filename),
    "encoded_style": encoded_style_noisy
}
result = demo_32x.run(**kwargs)
display_result(result).

Upscaling...
Done.


interactive(children=(Dropdown(description='Visualize:', index=1, options=('encoded_style', 'fake_image', 'ima…